In [ ]:
%%capture
!pip install np_utils



In [ ]:
import numpy as np
import librosa
from sklearn.model_selection import train_test_split,KFold,StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder,normalize
import pickle

import keras
from keras.layers import BatchNormalization

from keras.callbacks import ModelCheckpoint
from keras.layers import  MaxPool2D,Dense,Input,Convolution2D,Flatten,Activation,Dropout
from keras import models,optimizers,losses
from keras.utils import np_utils


from keras.activations import softmax, relu 


In [ ]:
from google.colab import drive
import os
import sys
drive.mount('/content/drive')
os.chdir("./drive/MyDrive/Project/")





Mounted at /content/drive


In [ ]:
lab = ['01_neutral','02_calm','03_happy','04_sad','05_angry','06_fearful','07_disgust','08_surprised']

In [ ]:
def convert_mfcc(path, bands, sample,duration):
    mf = np.empty(shape=(len(path), bands, 282, 1))
    total_len = sample*duration
    for i,one_path in enumerate(path):
        data,sr = librosa.load(one_path,sr=sample,res_type="kaiser_fast",duration=duration,offset=0.8) # offeset is the start
        data_len = len(data)
        if data_len > total_len:
            offset = np.random.randint((data_len - total_len))
            data = data[offset:(total_len+offset)]
        elif total_len > data_len:
            offset = np.random.randint(( total_len-data_len))
            data = np.pad(data, (offset, int(total_len) - len(data) - offset), "constant")
        else:
            data = np.pad(data, (0, int(total_len) - len(data)), "constant")
        mf[i,] = np.expand_dims(librosa.feature.mfcc(data, sr=sample, n_mfcc=bands), axis=-1)
    return mf
def get_model(bands):
    input = Input(shape=(bands,282,1))
    X = Convolution2D(64, (10,30), padding="same")(input)
    X = BatchNormalization()(X)
    X = Activation("relu")(X)
    X = MaxPool2D()(X)
    X = Dropout(rate=0.2)(X)
    
    X = Convolution2D(64, (10,30), padding="same")(X)
    X = BatchNormalization()(X)
    X = Activation("relu")(X)
    X = MaxPool2D()(X)
    X = Dropout(rate=0.2)(X)

    
    X = Convolution2D(64, (10,30), padding="same")(X)
    X = BatchNormalization()(X)
    X = Activation("relu")(X)
    X = MaxPool2D()(X)
    X = Dropout(rate=0.75)(X)

    X = Flatten()(X)
    X = Dense(64)(X)
    X = Dropout(rate=0.2)(X)
    X = BatchNormalization()(X)
    X = Activation("relu")(X)
    X = Dropout(rate=0.2)(X)
    
    out = Dense(8, activation=softmax)(X)
    model = models.Model(inputs=input, outputs=out)
    model.compile(optimizer=optimizers.Adam(0.001), loss=losses.categorical_crossentropy, metrics=['acc'])
    return model


In [ ]:
wav_list = []
labels = []
for i in range(1,25):
    da= os.listdir(f"Data/Actor_%02d"% (i,))
    # print(da[0])
    
    for k in da:
        if '.wav' in k:
            labels.append(k[6:8])
            wav_list.append(f"./Data/Actor_%02d/"% (i,) + k) 
            # break




In [ ]:
sample=48000
duration=3
bands = 30
mfcc = convert_mfcc(wav_list, bands,sample,duration)

In [ ]:
# with open('mfcc_rav_1.pkl', 'wb') as handle:
#     pickle.dump(mfcc, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open('label_rav_1.pkl', 'wb') as handle:
#     pickle.dump(labels, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
mfcc = []
with open('mfcc_rav_1.pkl', 'rb') as handle:
    mfcc = pickle.load(handle)
label = []
with open('label_rav_1.pkl', 'rb') as handle:
    labels = pickle.load(handle)
n_mfcc = 30

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(mfcc
                                                    , labels
                                                    , test_size=0.2
                                                    , shuffle=True
                                                    , random_state=42
                                                    ,stratify=labels
                                                   )



In [ ]:


lb = LabelEncoder()
y_train = np_utils.to_categorical(lb.fit_transform(y_train))
y_test = np_utils.to_categorical(lb.fit_transform(y_test))


In [ ]:


mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)

X_train = (X_train - mean)/std
X_test = (X_test - mean)/std

checkpoint = ModelCheckpoint(filepath='e.h5', 
                             monitor='val_acc',
                             verbose=1, 
                             save_best_only=True,
                             mode='max')

model = get_model(bands)
model_history = model.fit(X_train, y_train, validation_data=(X_test, y_test), 
                    batch_size=64, verbose = 2, epochs=400,callbacks=[checkpoint])


In [ ]:
# model.load_weights('e.h5')


In [ ]:
%%capture cap --no-stderr
# KF = KFold(n_splits=5, random_state=42, shuffle=True)
KF = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)



for train_index, test_index in KF.split(mfcc,labels):
    checkpoint = ModelCheckpoint(filepath='e.h5', 
                             monitor='val_acc',
                             verbose=1, 
                             save_best_only=True,
                             mode='max')
    X_train, X_test = mfcc[train_index], mfcc[test_index]
    y_train, y_test = np.asarray(labels)[train_index], np.asarray(labels)[test_index]

    lb = LabelEncoder()
    y_train = np_utils.to_categorical(lb.fit_transform(y_train))
    y_test = np_utils.to_categorical(lb.fit_transform(y_test))

    mean = np.mean(X_train, axis=0)
    std = np.std(X_train, axis=0)

    X_train = (X_train - mean)/std
    X_test = (X_test - mean)/std

    model = get_model(bands)
    model_history = model.fit(X_train, y_train, validation_data=(X_test, y_test), 
                        batch_size=64, verbose = 2, epochs=400,callbacks=[checkpoint]) 
    
    



In [ ]:
cap

In [ ]:
with open('output.txt', 'w') as f:
    f.write(cap.stdout)

In [ ]:
# model.load_weights('e.h5')
